In [9]:
import numpy as np
import cv2
import basic_image_operations as basic_ops
import filters
import geometrical_image_operations as geo_ops
import traffic_sign_detection as tsd
import colors
import numpy as np
import statistical_operations as stat_ops

import numpy as np
import cv2

def my_find_contours(binary_img, return_info=False, min_area=1, angle_tolerance=10):
    visited = np.zeros_like(binary_img, dtype=bool)
    contours = []
    contour_info = []

    rows, cols = binary_img.shape

    directions = [(-1, 0), (-1, 1), (0, 1), (1, 1),
                  (1, 0), (1, -1), (0, -1), (-1, -1)]

    def is_valid(y, x):
        return 0 <= y < rows and 0 <= x < cols

    def is_edge(y, x):
        if binary_img[y, x] != 255:
            return False
        for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            ny, nx = y + dy, x + dx
            if is_valid(ny, nx) and binary_img[ny, nx] == 0:
                return True
        return False

    def trace_contour(y, x):
        contour = []
        stack = [(y, x)]
        visited[y, x] = True

        while stack:
            cy, cx = stack.pop()
            contour.append([cx, cy])
            for dy, dx in directions:
                ny, nx = cy + dy, cx + dx
                if is_valid(ny, nx) and not visited[ny, nx] and is_edge(ny, nx):
                    visited[ny, nx] = True
                    stack.append((ny, nx))
                    break
        return np.array(contour).reshape(-1, 1, 2)

    for y in range(1, rows - 1):
        for x in range(1, cols - 1):
            if is_edge(y, x) and not visited[y, x]:
                contour = trace_contour(y, x)
                area = len(contour)
                if area < min_area:
                    continue

                # Check for diamond orientation
                rect = cv2.minAreaRect(contour)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                _, (w, h), angle = rect

                # Normalize angle: OpenCV may give values like -45 or 135
                angle = abs(angle % 180)
                angle = 90 - angle if angle > 90 else angle  # normalize to 0–90°

                if abs(angle - 45) > angle_tolerance:
                    continue  # Not a diamond shape

                contours.append(contour)

                if return_info:
                    x_vals = contour[:, 0, 0]
                    y_vals = contour[:, 0, 1]
                    x_min, y_min = x_vals.min(), y_vals.min()
                    x_max, y_max = x_vals.max(), y_vals.max()
                    bbox = (x_min, y_min, x_max - x_min, y_max - y_min)
                    contour_info.append({
                        'bbox': bbox,
                        'area': area,
                        'angle': angle
                    })

    if return_info:
        return contours, contour_info
    return contours

In [10]:
img = basic_ops.load_image(image_path='traffic_sign_images/vfs/vfs_01.jpg')
img = geo_ops.resize_image(image=img, target_height=img.shape[0]//8, target_width=img.shape[1]//8)
img = filters.median_filter(image=img, dim=3)
img = filters.gray_scale_filter(image=img)
img = filters.sobel_filter(image=img, mode='both')
img = filters.black_white_filter(image=img, threshold=40)
basic_ops.show_image(image=img)

Image loaded from traffic_sign_images/vfs_01.jpg
Image displayed


In [11]:

contours, info = my_find_contours(img, return_info=True, min_area=100, angle_tolerance=10)
output = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

for contour, data in zip(contours, info):
    x, y, w, h = data['bbox']
    area = data['area']
    cv2.drawContours(output, [contour], -1, (0, 255, 0), 1)
    cv2.rectangle(output, (x, y), (x + w, y + h), (255, 0, 0), 1)
    cv2.putText(output, f"{area}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)

basic_ops.show_image(image=output, title='output')

C:\Users\Crusader\AppData\Local\Temp\ipykernel_35268\521426312.py:63: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


Image displayed
